In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
import numpy as np
import math

In [5]:
df = pd.read_csv('Nat_Gas.csv', parse_dates=['Dates'], dayfirst=True)
df.rename(columns={'Dates': 'ds', 'Prices': 'y'}, inplace=True)

df.head()

model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
model.fit(df)

future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

def estimate_price(date_str):
    # Convert string date to datetime
    date = pd.to_datetime(date_str, dayfirst=True)
    # Create a dataframe for the input date
    future_date_df = pd.DataFrame({'ds': [date]})
    # Predict the price
    forecast_date = model.predict(future_date_df)
    return forecast_date['yhat'].values[0]

16:12:40 - cmdstanpy - INFO - Chain [1] start processing
16:12:40 - cmdstanpy - INFO - Chain [1] done processing


In [17]:
from datetime import date

def price_contract(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, max_volume, injection_withdrawal_cost_rate):
    volume = 0
    buy_cost = 0
    cash_in = 0

    # Combine injection and extraction dates
    all_dates = sorted(set(in_dates + out_dates))

    for current_date in all_dates:
        # Injection
        if current_date in in_dates:
            if volume < max_volume:
                volume += rate
                buy_cost += rate * in_prices[in_dates.index(current_date)]  # Cost to purchase gas
                buy_cost += rate * injection_withdrawal_cost_rate  # Injection cost
                print(f'Injected gas on {current_date} at a price of {in_prices[in_dates.index(current_date)]}')
            else:
                print(f'Injection is not possible on date {current_date} as there is insufficient space in the storage facility')

        # Extraction
        elif current_date in out_dates:
            if volume >= rate:
                volume -= rate
                cash_in += rate * out_prices[out_dates.index(current_date)]  # Revenue from selling gas
                cash_in -= rate * injection_withdrawal_cost_rate  # Withdrawal cost
                print(f'Extracted gas on {current_date} at a price of {out_prices[out_dates.index(current_date)]}')
            else:
                print(f'Extraction is not possible on date {current_date} as there is insufficient volume of gas stored')

    # Calculate storage costs
    storage_cost = math.ceil((max(out_dates) - min(in_dates)).days / 30) * storage_cost_rate
    return cash_in - storage_cost - buy_cost

In [22]:
# Example usage of price_contract()

in_dates = [date(2022, 6, 1), date(2022, 7, 1), date(2022, 8, 21), date(2022, 8, 1)]  # Injection dates
in_prices = []  # Prices on the injection days

for x in in_dates:
    price = estimate_price(x)
    in_prices.append(price)

    
out_dates = [date(2023, 1, 27), date(2022, 12, 15), date(2023, 1, 20), date(2023, 12, 1)]  # Extraction dates
out_prices = [] 

for x in out_dates:
    price = estimate_price(x)
    out_prices.append(price)
    
    
rate = 100000  # Rate of gas in cubic feet per day
max_storage_volume = 500000  # Maximum storage capacity of the storage facility
storage_cost_rate = 10000  # Storage cost rate per month
injection_withdrawal_cost_rate = 0.0005  # $/cf

result = price_contract(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, max_storage_volume, injection_withdrawal_cost_rate)
print(f"The value of the contract is: ${result}")

Injected gas on 2022-06-01 at a price of 10.063506358770557
Injected gas on 2022-07-01 at a price of 10.617499974838074
Injected gas on 2022-08-01 at a price of 10.462936246466008
Injected gas on 2022-08-21 at a price of 8.182685691006007
Extracted gas on 2022-12-15 at a price of 10.814865073729425
Extracted gas on 2023-01-20 at a price of 12.317479747768363
Extracted gas on 2023-01-27 at a price of 12.21650792125915
Extracted gas on 2023-12-01 at a price of 12.122519628717953
The value of the contract is: $624074.4100394249
